![image](https://www.ewatercycle.org/assets/logo.png)

# Case study -1: Hellow world. Generate forcings and run Marrmot model

This notebook demostractes how to use a model in eWatercycle by showing how to run a model on a catchment and forcing data (ERA5). This notebook also contains the steps for the preprocessing of the forcing data into model specific datasets using ESMValTool.

The model used in this notebook is:

* Marrmot M01 Collie River 1 (traditional bucket)


## Import statements
We'll be using the following modules

In [ ]:
from datetime import datetime

from cftime import num2date

from ewatercycle import forcing
from ewatercycle.models import MarrmotM01
from ewatercycle.observation.grdc import get_grdc_data
from ewatercycle.analysis import hydrograph

### Directory settings

### Experiment settings

In [ ]:
dataset = "ERA5"
start_date = datetime(2001, 1, 1)
end_date = datetime(2002, 1, 15)
shapefile = "Merrimack/Merrimack.shp"

# For observations & hydrograph
grdc_station_id = "4147380"

### Model specific settings

In [ ]:
# Values based on existing calibration
maximum_soil_moisture_storage = 1999.0
initial_soil_moisture_storage = 0.9 * maximum_soil_moisture_storage
discharge_variable = "flux_out_Q"
# flux_out_Q unit conversion factor from mm/day to m3/s
conversion_mmday2m3s = 1 / (1000 * 86400)

## Derived constants

In [ ]:
# project directories for model input i.e. shape files and forcing
shape = shapereader.Reader(shapefile)
attr = next(shape.records())
merrimack_area = attr.attributes["SUB_AREA"] * 1e6
print("The catchment area is:", merrimack_area)

### Pre processing model forcing using ESMValTool

In [ ]:
forcing_output = forcing.generate(
    model='marrmot',
    dataset=dataset,
    startyear=start_date.year,
    endyear=end_date.year,
    shapefile=shapefile
)
forcing_output

### Generate Model container

In [ ]:
model = MarrmotM01()
cfg_file, cfg_dir = model.setup(
    forcing=forcing_output,
    maximum_soil_moisture_storage=maximum_soil_moisture_storage,
    initial_soil_moisture_storage=initial_soil_moisture_storage,
)
cfg_file, cfg_dir

### Run model & capture discharge

In [ ]:
model.initialize(str(cfg_file))

In [ ]:
time_units = model.get_time_units()
timestamps = []
output = []

In [ ]:
while (model.get_current_time() < model.get_end_time()):
    model.update()
    current_time = num2date(
        model.get_current_time(),
        time_units,
        only_use_cftime_datetimes=False
    )
    timestamps.append(current_time)
    # Marrmot M01 is a lumped model, so only single value is returned
    value_in_mmday = model.get_value(discharge_variable)[0]
    # Convert from mm/day to m3/s
    value = value_in_mmday * merrimack_area * conversion_mmday2m3s
    output.append(value)

In [ ]:
# Show last 10 values
zip(timestamps, output)[-10:]

### Load observations

In [ ]:
observations = get_grdc_data(
    station_id=grdc_station_id,
    start_date=start_date.strftime("%Y-%m-%d"),
    end_date=end_date.strftime("%Y-%m-%d"),
)
observations

### Plot hydrograph

In [3]:
# TODO combine model output and observations into single dataframe
discharge = ???

In [ ]:
hydrograph(
    discharge=discharge,
    reference='observation',
)